In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import sys 
import asyncio

#Fix Windoiws issues in jupyter notebooks
if sys.platform == "win32":
    # 1. Use ProactorEventLoop for subprocess support
    if not isinstance(asyncio.get_event_loop_policy(),asyncio.WindowsProactorEventLoopPolicy):
        asyncio.set_event_loop_policy(asyncio.WindowsProactorEventLoopPolicy())
    
    # 2. Redirectstderr to avoid fileno() error when launching MCP servers
    if "ipykernel" in sys.modules:
        sys.stderr=sys.__stderr__

# Local MCP Server

In [3]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client=MultiServerMCPClient(
    {
        "local_server":{
            "transport":"stdio",
            "command":"python",
            "args":["C:/Users/SHUBHAM SARKAR/Desktop/Agentic/lca-lc-foundations/notebooks/module-2/resources/2.1_mcp_server.py"],
        }
    }
)

In [4]:
# get tools
tools = await client.get_tools()

# get resources
resources = await client.get_resources("local_server")

# get prompts
prompt = await client.get_prompt("local_server", "prompt")
prompt = prompt[0].content

In [5]:
from langchain.agents import create_agent
agent=create_agent(model="gpt-5-nano",tools=tools,system_prompt=prompt)

In [6]:
from langchain.messages import HumanMessage

config={"configurable":{"thread_id":"1"}}

response= await agent.ainvoke(
    {"messages":[HumanMessage(content="tell me about the langchain-mcp-adapters library")]},
    config=config
)

In [7]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='tell me about the langchain-mcp-adapters library', additional_kwargs={}, response_metadata={}, id='6cc19092-a6e0-42b2-99a5-adfcb6010b39'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 284, 'prompt_tokens': 271, 'total_tokens': 555, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CrfbeRVU8ycA3IJ0mWbCmF4z6XytY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b63e2-96fc-7eb0-bdbc-22ac9f35fcb8-0', tool_calls=[{'name': 'search_web', 'args': {'query': 'langchain-mcp-adapters'}, 'id': 'call_DbuiPahxxb5hRgC1ap0Y6doR', 'type': 'tool_call'}], usage_metadata={'input_to

# Online MCP 

In [8]:
client = MultiServerMCPClient(
    {
        "time": {
            "transport": "stdio",
            "command": "uvx",
            "args": [
                "mcp-server-time",
                "--local-timezone=America/New_York"
            ]
        }
    }
)

tools = await client.get_tools()

In [9]:
agent = create_agent(
    model="gpt-5-nano",
    tools=tools,
)

In [10]:
question = HumanMessage(content="What time is it?")

response = await agent.ainvoke(
    {"messages": [question]}
)

pprint(response)

{'messages': [HumanMessage(content='What time is it?', additional_kwargs={}, response_metadata={}, id='4751187a-3a2d-430a-867d-0b1513bb4c18'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 561, 'prompt_tokens': 296, 'total_tokens': 857, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CrfcTf12mpDJZRynUZpItkb59eBsf', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019b63e3-622d-7283-91a6-34b34be087c1-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'America/New_York'}, 'id': 'call_ytqzHb0QqF2Bf9oyAzPe2obJ', 'type': 'tool_call'}, {'name': 'get_current_time', 'args': {'timezone': 'Euro